In [1]:
from validphys.api import API
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from validphys.fkparser import load_fktable
from n3fit.layers.rotations import FlavourToEvolution
from collections import defaultdict

from typing import List, Dict
from collections import namedtuple

Using Keras backend


In [2]:
from numpy.typing import ArrayLike

In [3]:
seed = 1341351341

In [4]:
def generate_sequential_model(outputs=1, 
                   input_layer=None, 
                   nlayers=2, 
                   units=[100,100],
                   seed=seed,
                   **kwargs):
  """
  Create a tensorflow sequential model where all intermediate layers have the same size
  This function accepts an already constructed layer as the input.

  All hidden layers will have the same number of nodes for simplicity

  Arguments:
      outputs: int (default=1)
          number of output nodes (how many flavours are we training)
      input_layer: KerasTensor (default=None)
          if given, sets the input layer of the sequential model
      nlayers: int
          number of hidden layers of the network
      units: int
          number of nodes of every hidden layer in the network
      activation: str
          activation function to be used by the hidden layers (ex: 'tanh', 'sigmoid', 'linear')
  """
  if len(units) != nlayers:
      raise Exception("The length of units must match the number of layers.")
  
  if kwargs.get('kernel_initializer'):
      kernel_initializer = kwargs['kernel_initializer']
  else:
      kernel_initializer = tf.keras.initializers.HeNormal

  if kwargs.get('activation_list'):
      activation_list = kwargs['activation_list']
      if len(units) != len(activation_list):
          raise Exception("The length of the activation list must match the number of layers.")
  else:
      activation_list = ['tanh' for _ in range(nlayers)]

  if kwargs.get('output_func'):
      output_func = kwargs['output_func']
  else:
      output_func = 'linear'
  
  if kwargs.get('name'):
      name = kwargs['name']
  else:
      name = 'pdf'
  
  model = tf.keras.models.Sequential(name=name)
  if input_layer is not None:
      model.add(input_layer)
  for layer in range(nlayers):
      model.add(tf.keras.layers.Dense(units[layer], 
                                      activation=activation_list[layer],
                                      kernel_initializer=kernel_initializer(seed=seed - layer),
                                      ),
      )
  model.add(tf.keras.layers.Dense(outputs, 
                                  activation=output_func, 
                                  kernel_initializer=kernel_initializer(seed=seed - nlayers)
                                  ))

  return model

def compute_ntk(model, input):
  # Record operations for gradient computation
  batch_size = input.size
  n_outputs = model.layers[-1].units
  x = tf.convert_to_tensor(input)
  x = tf.reshape(x, shape=(-1,1))
  with tf.GradientTape(persistent=True) as tape:
      tape.watch(x)
      # Forward pass
      predictions = model(x)

  jacobian = tape.jacobian(predictions, model.trainable_variables)  

  ntk = tf.zeros((n_outputs, batch_size, n_outputs, batch_size))
  for jac in jacobian:
      jac_concat = tf.reshape(jac, (jac.shape[1], jac.shape[0], np.prod(jac.shape[2:]))) 
      ntk += tf.einsum('iap,jbp->iajb', jac_concat, jac_concat)  
  return ntk


def compute_mr(A : ArrayLike, 
               C : ArrayLike,
               Gamma : ArrayLike = None, 
               eta : float = None):
    '''Compute the tensor M that defines the inverse problem.
       The tensor can be regularised by means of the tensor gamma.
       THe magnitude if the regularization is specified by the scalar eta.
    '''
    M = np.tensordot(C, A, axes=[[1],[0]])
    M = np.tensordot(A, M, axes=[[0],[0]])
    R = np.zeros_like(M)
    if Gamma is not None and eta is not None:
        R = eta * np.tensordot(Gamma, Gamma, axes=[[0],[0]])
    return M + R

In [5]:
dataset_inputs = [
  #{'dataset': 'NMC_NC_NOTFIXED_DW_EM-F2', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'NMC_NC_NOTFIXED_P_EM-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'SLAC_NC_NOTFIXED_P_DW_EM-F2', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'SLAC_NC_NOTFIXED_D_DW_EM-F2', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'BCDMS_NC_NOTFIXED_P_DW_EM-F2', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'BCDMS_NC_NOTFIXED_D_DW_EM-F2', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'CHORUS_CC_NOTFIXED_PB_DW_NU-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'CHORUS_CC_NOTFIXED_PB_DW_NB-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'NUTEV_CC_NOTFIXED_FE_DW_NU-SIGMARED', 'cfac': ['MAS'], 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'NUTEV_CC_NOTFIXED_FE_DW_NB-SIGMARED', 'cfac': ['MAS'], 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_NC_318GEV_EM-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_NC_225GEV_EP-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_NC_251GEV_EP-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_NC_300GEV_EP-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_NC_318GEV_EP-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_CC_318GEV_EM-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_CC_318GEV_EP-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_NC_318GEV_EAVG_CHARM-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
  {'dataset': 'HERA_NC_318GEV_EAVG_BOTTOM-SIGMARED', 'frac': 0.75, 'variant': 'legacy'},
]

In [6]:
common_dict = dict(
    dataset_inputs=dataset_inputs,
    metadata_group="nnpdf31_process",
    use_cuts='internal',
    datacuts={'t0pdfset': '240701-02-rs-nnpdf40-baseline', 'q2min': 3.49, 'w2min': 12.5},
    theoryid=40000000
)

In [7]:
groups_data = API.procs_data(**common_dict)
groups_index = API.groups_index(**common_dict)
C = API.groups_covmat_no_table(**common_dict)
Cinv = np.linalg.inv(C)
Cinv = pd.DataFrame(Cinv, index=C.index, columns=C.columns)

In [8]:
fk_table_list = defaultdict(list)
x_grid_list = defaultdict(list)
central_values = defaultdict(list)
Y = defaultdict(list)

total_ndata_wc = 0
total_grid_size = 0
start_grid_by_exp = defaultdict(list)
grid_size_by_exp = defaultdict(list)
start_proc_by_exp = defaultdict(list)
M_list = []

for idx_proc, group_proc in enumerate(groups_data):
  for idx_exp, exp_set in enumerate(group_proc.datasets):

    fkspecs = exp_set.fkspecs
    cuts = exp_set.cuts
    ndata = exp_set.load_commondata().ndata
    fk_table = load_fktable(fkspecs[0])
    fk_table_wc = fk_table.with_cuts(cuts)
    x_grid = fk_table_wc.xgrid
    fk_table_wc_np = fk_table_wc.get_np_fktable()

    Y[exp_set.name] = exp_set.load_commondata().with_cuts(cuts).central_values.to_numpy()
    fk_table_list[exp_set.name] = fk_table_wc_np
    x_grid_list[exp_set.name] = x_grid
    start_proc_by_exp[exp_set.name] = total_ndata_wc
    start_grid_by_exp[exp_set.name] = total_grid_size
    grid_size_by_exp[exp_set.name] = x_grid.shape[0]
    total_grid_size += x_grid.shape[0]
    total_ndata_wc += ndata

In [9]:
evol_basis_pids = tuple(
      [22, 100, 21, 200]
      + [200 + n**2 - 1 for n in range(2, 6 + 1)]
      + [100 + n**2 - 1 for n in range(2, 6 + 1)]
  )

In [10]:
evol_basis_pids

(22, 100, 21, 200, 203, 208, 215, 224, 235, 103, 108, 115, 124, 135)

In [11]:
for idx, c in enumerate(grid.channels()):
    print(f"{idx}: {c}")

NameError: name 'grid' is not defined

In [405]:
flav_map = {
        0: "g",
        1: "t3",
        2: "t8",
        3: "t15 (c-)",
        4: "sigma",
        5: "v3",
        6: "v8",
        7: "v15",
        8: "v",}

# Test with one single DIS data set

In [ ]:
fk_table_list.keys()

In [407]:
expname = 'HERA_NC_225GEV_EP-SIGMARED'
fk_exp = fk_table_list[expname]
Cinv_exp = Cinv.xs(level='dataset', key=expname).T.xs(level='dataset', key=expname)
xgrid_exp = x_grid_list[expname]
y_exp = Y[expname]

The NNPDF-like model

In [408]:
# Generate NNPDF model
nnpdf = generate_sequential_model(outputs=9, nlayers=2, units=[28, 20], seed=seed, name='NNPDF', kernel_initializer=tf.keras.initializers.GlorotNormal)

The NTK in the grid space

In [417]:
# Compute the ntk
ntk_exp = compute_ntk(nnpdf, xgrid_exp).numpy()
ntk = ntk_exp.reshape(ntk_exp.shape[0] * ntk_exp.shape[1], -1)

In [418]:
# Compute the matrix M
Cinv_fk = np.tensordot(Cinv_exp, fk_exp, axes=[[1],[0]])
M = np.tensordot(fk_exp, Cinv_fk, axes=[[0],[0]])
M = M.reshape(M.shape[0] * M.shape[1], -1)

In [465]:
# Implicit regularisation of GD
eta_gd = 0.001

# Implicit regularisation from gradient descent
#ntk_M = np.tensordot(ntk_exp, M, axes=[[2,3],[0,1]])
#M_ntk_M = np.tensordot(M, ntk_M, axes=[[0,1],[0,1]])
#M_ntk_M.reshape(M_ntk_M.shape[0] * M_ntk_M.shape[1], -1)

M_gd = eta_gd * ( M.T @ ntk @ M + 1)

In [468]:
# Identity regularisation
eta_id = 0.001
M_id = eta_id * np.identity(M.shape[0])

In [ ]:
# Invert the matrix
Mr = M - M_gd
np.linalg.inv(Mr)

In [410]:
#eta_gd = 0
#eta_np_reg = 0.001
#
## Implicit regularisation from gradient descent
#ntk_M = np.tensordot(ntk_exp, M, axes=[[2,3],[0,1]])
#M_ntk_M = np.tensordot(M, ntk_M, axes=[[0,1],[0,1]])
#M_ntk_M.reshape(M_ntk_M.shape[0] * M_ntk_M.shape[1], -1)
#Mr_exp = M - eta_gd * M_ntk_M
#
## flavour regularisation through identity
##Mr_exp_id = np.zeros_like(Mr_exp)
##for alpha in range(Mr_exp.shape[0]):
##  for beta in range(Mr_exp.shape[2]):
##    Id = eta_id * np.identity(M.shape[1])
##    Mr_exp_id[alpha, : , beta, :] = Mr_exp[alpha, : , beta, :] + Id
#
## Regularisation numpy-like
#prod = 1
#shape = Mr_exp.shape
#ind = 2
#invshape = shape[ind:] + shape[:ind]
#for k in shape[ind:]:
#  prod *= k
#a = Mr_exp.reshape(prod, -1)
#Id = np.identity(prod)
#a = a + eta_np_reg * Id
#ia = np.linalg.inv(a)
#Mr_inv = ia#.reshape(*invshape)
#Mr_collapsed = a

In [ ]:
Mr_exp_tensor_inv = np.linalg.tensorinv(Mr_id_reg, ind=2)
# Check if it corresponds to the actual inverse
rng = np.random.default_rng()
b = rng.normal(size=(9, 26))
np.allclose(np.tensordot(Mr_exp_tensor_inv, b, axes=[[2,3], [0,1]]), np.linalg.tensorsolve(Mr_id_reg, b))

Compact the tensor $M$ into a matrix and compute the eigenvalues (useful?).

In [ ]:
M_compact = Mr_id_reg.reshape(prod,-1)
eigvals, eigvecs = np.linalg.eig(M_compact)
for i, val in enumerate(eigvals):
  print(f'lambda_{i+1} = {val.real} + i {val.imag}')

In [ ]:
for alpha in range(Mr_id_reg.shape[0]):
  for beta in range(Mr_id_reg.shape[2]):
    eigvals, eigvecs = np.linalg.eig(Mr_id_reg[alpha,:,beta,:])
    #eigvals = eigvals.real
    print(f'Eigvalues for flavour combination ({flav_map[alpha]}, {flav_map[beta]})')
    print(f"{np.sort(eigvals)}")
    print('___________________________\n')

## Check if the matrixes for each flavour are invertible

In [312]:
from numpy.linalg import LinAlgError

Mr_exp_inv = np.zeros_like(Mr_exp)
for alpha in range(Mr_exp.shape[0]):
  for beta in range(Mr_exp.shape[2]):
    try:
      Mr_exp_inv[alpha, : , beta, :] = np.linalg.inv(Mr_exp_id[alpha,:,beta,:])
    except LinAlgError:
      print(f"({alpha}, {beta}) not invertible")
    

In [313]:
#Mr_exp_inv = np.linalg.tensorinv(Mr_exp, 2)
#delta = np.tensordot(Mr_exp_inv, Mr_exp, axes=[[2,3], [0,1]])
#print(delta[4,:,4,:].diagonal())

Testing that the inverse is really what it claims to be

Constructing $H = \Theta M_R$

In [380]:
# collapse ntk
prod = 1
shape = ntk_exp.shape
ind = 2
invshape = shape[ind:] + shape[:ind]
for k in shape[ind:]:
  prod *= k
ntk_exp = ntk_exp.reshape(prod, -1)

In [381]:
#H = np.tensordot(ntk_exp, Mr_exp, axes=[[2,3],[0,1]])
H = Mr_collapsed @ ntk_exp

Constructing $D$, defined as
$$
D = M_R^{-1} A^T C_Y^{-1} y - f_0 = K - f_0 \,.
$$

In [382]:
# Computing K
Cinv_y = np.tensordot(Cinv_exp, y_exp, axes=[[1],[0]])
A_Cinv_y = np.tensordot(fk_exp, Cinv_y, axes=[[0],[0]])

In [383]:
A_Cinv_y = A_Cinv_y.reshape(-1)

In [384]:
K = Mr_inv @ A_Cinv_y
#K = np.tensordot(Mr_exp_inv, A_Cinv_y, axes=[[2,3], [0, 1]])

In [385]:
# Computing f0
f0 = nnpdf(xgrid_exp).numpy()
f0 = f0.reshape(-1)

Constructing the integrated solution
$$
f(t) = e^{-Ht} f_0 + \left( I -  e^{-Ht} \right) K \,.
$$
Since $H$ is a rank-4 tensor, I'll write the expression above as follows
$$
f^{\alpha}(t) = \sum_{\beta = 1}^{N_f} \left[ \left( e^{-Ht} \right)^{\alpha \beta} f_0^{\beta} + \left( I - e^{-Ht} \right)^{\alpha \beta} K^{\beta} \right]\,.
$$
Note that, at LO in the theory of wide neural networks, the NTK is constant. Furthermore, the NTK is diagonal in the flavours
$$
\Theta_{ij}^{\alpha \beta} = \delta^{\alpha \beta} \Theta_{ij} \,.
$$
The integrated solution diagonalises in the flavour space
$$
f^{\alpha}(t) = \left[ \left( e^{-Ht} \right)^{\alpha \alpha} f_0^{\alpha} + \left( I - e^{-Ht} \right)^{\alpha \alpha} K^{\alpha} \right]\,.
$$
The expression above is fully diagonalized. Furthermore, we can introduce the eigenspace of the operator $H$. Recall that the operator $H$ is diagonal in the flavour space, and the diagonalization is performed with respect to the data space
$$
H^{\alpha \alpha} v^{\alpha (k)} = \lambda_k^{\alpha} v^{\alpha (k)} \,.
$$
The vector $v^{\alpha (k)} \in \mathbb{R}^{N_{dat}}$ is the eigenvector relative to the $k$-th eigenvalue $\lambda_k^{\alpha}$ for the matrix $H$ specified by flavour $\alpha$. We can project the integrated equation in the eigenspace of $H$
$$
\begin{split}
f^{\alpha}(t) & = \sum_{k = 1}^{\textrm{eig}(H)} \left[  e^{- t \lambda^{\alpha}_k} \; \widetilde{f}_0^{\alpha (k)} + \left( 1 - e^{- t\lambda^{\alpha}_k} \right) \widetilde{K}^{\alpha(k)} \right] \; v^{(k) \alpha} \\
& = \sum_{k = 1}^{\textrm{H}(\Theta)} \left[  C_1^{\alpha(k)} + C_2^{\alpha(k)} \right] \; v^{(k) \alpha} \,,
\end{split}
$$
where
$$
C_1^{\alpha(k)} = e^{- t \lambda^{\alpha}_k} \; \widetilde{f}_0^{\alpha (k)} \\
C_2^{\alpha(k)} = \left( 1 - e^{- t\lambda^{\alpha}_k} \right) \widetilde{K}^{\alpha(k)}
$$
and $\widetilde{f}_0$, $\widetilde{K}$ are the component of the respective vectors in the eigenspace of $H$
$$
\widetilde{f}_0^{\alpha (k)} = \left< f_0^{\alpha}, v^{\alpha(k)} \right> \,,\\
\widetilde{K}^{\alpha (k)} = \left< K^{\alpha}, v^{\alpha(k)} \right> \,.
$$
Note that in general $H$ is a 4-rank tensor although we only consider the diagonal elements in the flavour. Effectively, as it can be seen from the equations above, this means that we are neglecting all the mixing contributions for different flavours. This is consistent with the fact that, for large networks, the output should are independent of each other at initialization. Furthermore, the integrated equation above extends the statement also during training, as flavour $\alpha$ does not receive any contribution from the other flavours.

In [387]:
def integrate_flow_t_v2(t):
  eigvals, eigvecs = np.linalg.eig(H)
  eigvals = eigvals.real
  eigvecs = eigvecs.real

  f0_tilde = [np.dot(f0, eigvecs[:, k]) for k in range(eigvals.size)]
  K_tilde = [np.dot(K, eigvecs[:, k]) for k in range(eigvals.size)]

  output = np.zeros(shape=K.shape[0])
  for k in range(eigvals.size):
      C1_k = f0_tilde[k] * np.exp(- eigvals[k] * t)
      C2_k = (1 - np.exp(- eigvals[k] * t) ) * K_tilde[k]
      output = np.add(output, (C1_k + C2_k) * eigvecs[:,k])

  return output

In [343]:

integrated_data = namedtuple('integrated_data', ['Coefficients', 'Eigenvectors', 'Eigenvalues'])

def integrate_flow_t(t):
  output_by_flavour = defaultdict(list)
  coeffs_eig_by_flavour = defaultdict(list)

  for alpha in range(f0.shape[1]):
    output = np.zeros(shape=ntk_exp.shape[1])
    for beta in range(f0.shape[1]):

      eigval, eigvec = np.linalg.eig(
        0.5*(H[alpha, :, beta, :] + H[alpha, :, beta, :].T)
             )
      print(eigval)
      eigval = abs(eigval.real)
      eigvec = eigvec.real

      
      f0_tilde = [np.dot(f0[:,beta], eigvec[:, k]) for k in range(eigval.size)]
      K_tilde = [np.dot(K[beta,:], eigvec[:, k]) for k in range(eigval.size)]
      
      #coefficients_collector = []
      
      for k in range(eigval.size):
        C1_k = f0_tilde[k] * np.exp(- eigval[k] * t)
        C2_k = (1 - np.exp(- eigval[k] * t) ) * K_tilde[k]
        output = np.add(output, (C1_k + C2_k) * eigvec[:,k])
        #coefficients_collector.append(C1_k + C1_k)
      
      #coefficients_eig = integrated_data(
      #  Coefficients=coefficients_collector,
      #  Eigenvectors=eigvec,
      #  Eigenvalues=eigval
      #)

      #coeffs_eig_by_flavour[flavour_map[alpha]] = coefficients_eig

    output_by_flavour[flavour_map[alpha]] = output

  return  output_by_flavour#, coeffs_eig_by_flavour

In [388]:
t = 100000
out = integrate_flow_t_v2(t)
out = out.reshape(shape[:2])

In [ ]:
import scipy.interpolate as scint
xnew = np.linspace(xgrid_exp.min(), xgrid_exp.max(), 300)  
interp = scint.CubicSpline(xgrid_exp, out[4])

fig, ax = plt.subplots(figsize=(10,6))
ax.plot(xnew, interp(xnew), color='green', label='Integrated solution')
#ax.set_title(f"Integrated solution after t = {t}")
ax.set_xlabel(r'$x$')
ax.set_xscale('log')
ax.set_ylabel(r'$T(t)$')
#ax.legend()